# Parte 5: Árboles de Decisión Potenciados

El paquete conifer se creó a partir de hls4ml, proporcionando un conjunto similar de características pero específicamente dirigido a la inferencia de Árboles de Decisión Potenciados. En este cuaderno entrenaremos un GradientBoostingClassifier con scikit-learn, utilizando el mismo conjunto de datos de etiquetado de jets que en los otros cuadernos de tutoriales. Luego convertiremos el modelo usando conifer y ejecutaremos la predicción y síntesis de precisión de bits como lo hicimos con hls4ml anteriormente.

`conifer` esta disponible en GitHub [aqui](https://github.com/thesps/conifer), y tenemos una publicación que describe la implementación y el rendimiento de la inferencia en detalle [aqui](https://iopscience.iop.org/article/10.1088/1748-0221/15/05/P05026/pdf).

<img src="https://github.com/thesps/conifer/blob/master/conifer_v1.png?raw=true" width="250" alt="conifer">

En este código, estamos importando las bibliotecas necesarias para trabajar con modelos de árboles de decisión mejorados mediante el paquete conifer. Utilizamos NumPy para operaciones numéricas, scikit-learn para entrenar el modelo de clasificación, joblib para guardar y cargar modelos entrenados, conifer para la conversión del modelo y plotting y matplotlib.pyplot para visualización. También configuramos la variable de entorno PATH para incluir la ruta de Vivado HLS y establecemos la semilla aleatoria np.random.seed(0) para garantizar la reproducibilidad de los resultados.

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
import joblib
import conifer
import plotting
import matplotlib.pyplot as plt
import os

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']
np.random.seed(0)

## Cargar dataset
Este código carga el conjunto de datos previamente descargado y guardado en archivos numpy. Los archivos contienen los conjuntos de datos de entrenamiento y prueba (características y etiquetas) junto con las clases.

- `X_train_val`: contiene las características del conjunto de datos de entrenamiento y validación.
- `X_test`: contiene las características del conjunto de datos de prueba.
- `y_train_val`: contiene las etiquetas del conjunto de datos de entrenamiento y validación.
- `y_test`: contiene las etiquetas del conjunto de datos de prueba.
- `classes`: contiene las clases del conjunto de datos.

Los argumentos `allow_pickle=True` son necesarios porque los archivos numpy contienen objetos Python.

In [ ]:
X_train_val = np.load('X_train_val.npy')
X_test = np.load('X_test.npy')
y_train_val = np.load('y_train_val.npy')
y_test = np.load('y_test.npy', allow_pickle=True)
classes = np.load('classes.npy', allow_pickle=True)

Necesitamos transformar las etiquetas de prueba desde los valores codificados en one-hot a etiquetas simples.

In [ ]:
le = LabelEncoder().fit(classes)
ohe = OneHotEncoder().fit(le.transform(classes).reshape(-1, 1))
y_train_val = ohe.inverse_transform(y_train_val.astype(int))
y_test = ohe.inverse_transform(y_test)

## Entrenar un `GradientBoostingClassifier`
Usando con 20 estimadores y una profundidad máxima de 3. El número de árboles de decisión será n_estimators * n_classes, por lo que será de 100 para este conjunto de datos. Si estás volviendo a este cuaderno después de haber entrenado el BDT una vez, establece train = False para cargar el modelo en lugar de volver a entrenarlo.

In [ ]:
train = True
if train:
    clf = GradientBoostingClassifier(n_estimators=20, learning_rate=1.0, max_depth=3, random_state=0, verbose=1).fit(
        X_train_val, y_train_val.ravel()
    )
    if not os.path.exists('model_5'):
        os.makedirs('model_5')
    joblib.dump(clf, 'model_5/bdt.joblib')
else:
    clf = joblib.load('model_5/bdt.joblib')

## Crea una configuración de conifer

Similarmente a hls4ml, podemos usar un método de utilidad para obtener una plantilla del diccionario de configuración que podemos modificar.

In [ ]:
cfg = conifer.backends.xilinxhls.auto_config()
cfg['OutputDir'] = 'model_5/conifer_prj'
cfg['XilinxPart'] = 'xcu250-figd2104-2L-e'
plotting.print_dict(cfg)

## Convert the model
La sintaxis para la conversión del modelo con conifer es un poco diferente a la de hls4ml. Construimos un objeto conifer.model, proporcionando el BDT entrenado, el convertidor correspondiente a la biblioteca que utilizamos, el "backend" de conifer al que deseamos apuntar y la configuración.

conifer tiene convertidores para:

- sklearn
- xgboost
- tmva

Y backends:

- vivadohls
- vitishls
- xilinxhls (usa cualquiera de vivado o vitis que esté en la ruta)
- vhdl

Aquí usaremos el convertidor de sklearn, ya que así es como entrenamos nuestro modelo, y el backend vivadohls. Para BDT más grandes con muchos más árboles o profundidad, puede ser preferible generar VHDL directamente usando el backend vhdl para obtener el mejor rendimiento. Consulta nuestro artículo para ver la comparación de rendimiento entre esos backends.

In [ ]:
cnf = conifer.model(clf, conifer.converters.sklearn, conifer.backends.vivadohls, cfg)
cnf.compile()

## perfil
Del mismo modo que en hls4ml, podemos visualizar la distribución de los parámetros del BDT para orientar la elección de la precisión.

In [ ]:
cnf.profile()

## Ejecutar inferencia
Ahora podemos realizar la inferencia del BDT con sklearn, y también la simulación exacta de bits utilizando Vivado HLS. La salida que produce el BDT de conifer es equivalente al método decision_function.

In [ ]:
y_skl = clf.decision_function(X_test)
y_cnf = cnf.decision_function(X_test)

## Checar el rendimiento
Imprime la precisión de las evaluaciones de sklearn y conifer, y traza las curvas ROC. Deberíamos ver que podemos llegar bastante cerca de la precisión de las redes neuronales de las partes 1-4.

In [ ]:
yt = ohe.transform(y_test).toarray().astype(int)
print("Accuracy sklearn: {}".format(accuracy_score(np.argmax(yt, axis=1), np.argmax(y_skl, axis=1))))
print("Accuracy conifer: {}".format(accuracy_score(np.argmax(yt, axis=1), np.argmax(y_cnf, axis=1))))
fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.makeRoc(yt, y_skl, classes)
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(yt, y_cnf, classes, linestyle='--')

## Sintetizar
Ahora ejecuta el paso de Síntesis en C de Vivado HLS para producir una IP que podamos usar, e inspecciona los recursos estimados y la latencia. Puedes ver alguna salida en vivo mientras se ejecuta la síntesis abriendo un terminal desde la página de inicio de Jupyter y ejecutando:
`tail -f model_5/conifer_prj/vivado_hls.log`

In [ ]:
cnf.build()

## Leer informe
Podemos utilizar una utilidad de hls4ml para leer el informe de Vivado.

In [ ]:
import hls4ml

hls4ml.report.read_vivado_report('model_5/conifer_prj/')